<H1> Classification de données textuelles </H1>


Lors de l'étape d'ingénierie de données textuelles nous avons vu que diverses opérations pouvaient être appliquées sur les textes et qu'au final il est possible d'obtenir des textes simplifiés. Nous allons, à présent, étudier comment faire de la classification à partir de données textuelles et comment convertir les textes en vecteurs pour pouvoir faire de la classification. 


## Vectorisation




Maintenant qu'un document a été transformé en une séquence de mot, il est nécessaire de la transformer en vecteur. C'est le rôle de la vectorisation.  



** Bag of Words **

La manière la plus simple de vectorisation est d'utiliser les Bag of Words (BOW). Il s'agit, à partir d'une liste de mots (vocabulaire) de compter le nombre d'apparition du mot du vocabulaire dans le document.  

Cette opération se fait par :
1. Création d'une instance de la classe CountVectorizer
1. Appel de la fonction fit() pour apprendre le vocabulaire à partir de document
1. Appel de la fonction transform() sur un ou plusieurs documents afin de les encoder dans le vecteur.  

Attention, par défaut, CountVectorizer effectue un certain nombre de pré-traitements comme par exemple mise en minuscule. Voir https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [1]:
from sklearn.feature_extraction.text import CountVectorizer

texte = ["This is an example of CountVectorizer for creating a vector"]

vectorizer = CountVectorizer()
# creation du vocabulaire
vectorizer.fit(texte)
# Contenu du vocabulaire
print(vectorizer.vocabulary_)
# encodage du document
vector = vectorizer.transform(texte)

print ("Taille du vecteur :\n",vector.shape)


{'this': 7, 'is': 5, 'an': 0, 'example': 3, 'of': 6, 'countvectorizer': 1, 'for': 4, 'creating': 2, 'vector': 8}
Taille du vecteur :
 (1, 9)


Il est donc à présent possible de traiter un ensemble de documents comme le montre l'exemple suivant. Nous créons également un dataframe.

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is my first document.',
    'This is the document 2 !',
    'Maybe this is the third document?',
    'Anything else? may be 40',
    'Yes !! this is the last one'
]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

# vectorizer.get_feature_names())
# contient le vocabulaire


df = pd.DataFrame(
    data=vectorizer.transform(corpus).toarray(),
    columns=vectorizer.get_feature_names()
)

display(df)

,40,anything,be,document,else,first,is,last,may,maybe,my,one,the,third,this,yes
0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,1,0
1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0
2,0,0,0,1,0,0,1,0,0,1,0,0,1,1,1,0
3,1,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,1,0,0,0,1,1,0,1,1


** Prise en compte des prétraitements  **  

Considérons les prétraitements suivants qui permettent de supprimer les caractères non Ascii, de mettre en minuscule, d'enlever les ponctuations, de remplacer les nombres et d'enlever les stopwords.

In [3]:
from nltk.tokenize import word_tokenize
import unicodedata
import re
import inflect
from nltk.corpus import stopwords

def remove_non_ascii(words):
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word) 
    return new_words

def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

def clean_text(text):
    tokens = word_tokenize(text)
    tokens=normalize(tokens)
    text="".join([" "+i for i in tokens]).strip()
    return text



In [4]:
texte="we have to think that is text is 1000 *#better than. the "
print ("Texte avant le nettoyage \n")
print (texte)
texte=clean_text(texte)
print ("Texte après le nettoyage \n")
print (texte)

Texte avant le nettoyage 

we have to think that is text is 1000 *#better than. the 
Texte après le nettoyage 

think text one thousand better


L'appel aux fonctions de prétraitements peut se faire directement dans CountVectorizer. Attention cependant il est préférable de ne pas le faire. Par exemple dans le cas d'un pipeline et d'un gridsearch le prétraitement sera effectué à chaque fois ! Il est par contre utile de le faire lors de la dernière étape et que le modèle est sauvegardé pour permettre qu'un nouveau document puisse être transformé avant d'être mis sous la forme d'un vecteur (voir plus bas).

In [5]:
print (len(corpus))
for i in range(len(corpus)):
    corpus[i]=clean_text(corpus[i])

5


In [6]:

#ici le preprocessor ne sert à rien
# car les données ont été nettoyées avant.
vectorizer = CountVectorizer(
    preprocessor=clean_text
)

X = vectorizer.fit_transform(corpus)


df = pd.DataFrame(
    data=vectorizer.transform(corpus).toarray(),
    columns=vectorizer.get_feature_names()
)

display(df)

,anything,document,else,first,forty,last,may,maybe,one,third,two,yes
0,0,1,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0,1,0,1,0,0
3,1,0,1,0,1,0,1,0,0,0,0,0
4,0,0,0,0,0,1,0,0,1,0,0,1


Appel dans CountVectorizer

In [7]:

corpus = [
    'This is my first document.',
    'This is the document 2 !',
    'Maybe this is the third document?',
    'Anything else? may be 40',
    'Yes !! this is the last one'
]
#Rappel ce n'est pas efficace
#il vaut mieux traiter les données avant
#attention aux pipelines
vectorizer = CountVectorizer(
    preprocessor=clean_text
)

X = vectorizer.fit_transform(corpus)


df = pd.DataFrame(
    data=vectorizer.transform(corpus).toarray(),
    columns=vectorizer.get_feature_names()
)

display(df)

,anything,document,else,first,forty,last,may,maybe,one,third,two,yes
0,0,1,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0,1,0,1,0,0
3,1,0,1,0,1,0,1,0,0,0,0,0
4,0,0,0,0,0,1,0,0,1,0,0,1


** TfidfVectorizer **

CountVectorizer en prenant en compte l'occurrence des mots est souvent trop limité. Une alternative est d'utiliser la 
mesure TF-IDF (Term Frequency – Inverse Document) via une instance de la classe TfidfVectorizer. Le principe est le même que pour CountVectorizer.

Remarque : Il est possible si CountVectorizer a déjà été utilisé de le faire suivre par TfidfTransformer pour simplement mettre à jour les valeurs.

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'This is my first document.',
    'This is the document 2 !',
    'Maybe this is the third document?',
    'Anything else? may be 40',
    'Yes !! this is the last one'
]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

# vectorizer.get_feature_names())
# contient le vocabulaire


df = pd.DataFrame(
    data=vectorizer.transform(corpus).toarray(),
    columns=vectorizer.get_feature_names()
)

display(df)

,40,anything,be,document,else,first,is,last,may,maybe,my,one,the,third,this,yes
0,0.000000,0.000000,0.000000,0.381399,0.000000,0.569497,0.320844,0.000000,0.000000,0.000000,0.569497,0.000000,0.000000,0.000000,0.320844,0.000000
1,0.000000,0.000000,0.000000,0.541107,0.000000,0.000000,0.455196,0.000000,0.000000,0.000000,0.000000,0.000000,0.541107,0.000000,0.455196,0.000000
2,0.000000,0.000000,0.000000,0.356359,0.000000,0.000000,0.299781,0.000000,0.000000,0.532109,0.000000,0.000000,0.356359,0.532109,0.299781,0.000000
3,0.447214,0.447214,0.447214,0.000000,0.447214,0.000000,0.000000,0.000000,0.447214,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.278803,0.494873,0.000000,0.000000,0.000000,0.494873,0.331422,0.000000,0.278803,0.494873


In [9]:
# Appel d'un pré-prétraiment
vectorizer = TfidfVectorizer(
    preprocessor=clean_text
)

X = vectorizer.fit_transform(corpus)


df = pd.DataFrame(
    data=vectorizer.transform(corpus).toarray(),
    columns=vectorizer.get_feature_names()
)

display(df)

,anything,document,else,first,forty,last,may,maybe,one,third,two,yes
0,0.0,0.556451,0.0,0.830881,0.0,0.00000,0.0,0.00000,0.00000,0.00000,0.000000,0.00000
1,0.0,0.556451,0.0,0.000000,0.0,0.00000,0.0,0.00000,0.00000,0.00000,0.830881,0.00000
2,0.0,0.427993,0.0,0.000000,0.0,0.00000,0.0,0.63907,0.00000,0.63907,0.000000,0.00000
3,0.5,0.000000,0.5,0.000000,0.5,0.00000,0.5,0.00000,0.00000,0.00000,0.000000,0.00000
4,0.0,0.000000,0.0,0.000000,0.0,0.57735,0.0,0.00000,0.57735,0.00000,0.000000,0.57735


Appel de fonction de prétraitement dans TfidfVectorizer

In [10]:
# Appel d'un pré-prétraiment
vectorizer = TfidfVectorizer(
    preprocessor=clean_text
)

X = vectorizer.fit_transform(corpus)


df = pd.DataFrame(
    data=vectorizer.transform(corpus).toarray(),
    columns=vectorizer.get_feature_names()
)

display(df)


,anything,document,else,first,forty,last,may,maybe,one,third,two,yes
0,0.0,0.556451,0.0,0.830881,0.0,0.00000,0.0,0.00000,0.00000,0.00000,0.000000,0.00000
1,0.0,0.556451,0.0,0.000000,0.0,0.00000,0.0,0.00000,0.00000,0.00000,0.830881,0.00000
2,0.0,0.427993,0.0,0.000000,0.0,0.00000,0.0,0.63907,0.00000,0.63907,0.000000,0.00000
3,0.5,0.000000,0.5,0.000000,0.5,0.00000,0.5,0.00000,0.00000,0.00000,0.000000,0.00000
4,0.0,0.000000,0.0,0.000000,0.0,0.57735,0.0,0.00000,0.57735,0.00000,0.000000,0.57735


** Les ngrams **

Très souvent il est utile de prendre en compte les n-grammes, i.e. la suite de n mots consécutifs car ils peuvent être important pour la classification. Il est tout à fait possible de les obtenir soit pendant l'étape de prétraitement, soit lors de l'étape de vectorisation en le passant en paramètre. 

In [11]:
# Appel d'un pré-prétraiment
vectorizer = TfidfVectorizer(
    preprocessor=clean_text,
    ngram_range=(1, 2)
)

X = vectorizer.fit_transform(corpus)


df = pd.DataFrame(
    data=vectorizer.transform(corpus).toarray(),
    columns=vectorizer.get_feature_names()
)

display(df)

,anything,anything else,document,document two,else,else may,first,first document,forty,last,...,may,may forty,maybe,maybe third,one,third,third document,two,yes,yes last
0,0.000000,0.000000,0.427993,0.00000,0.000000,0.000000,0.63907,0.63907,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
1,0.000000,0.000000,0.427993,0.63907,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.63907,0.000000,0.000000
2,0.000000,0.000000,0.317527,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.474125,0.474125,0.000000,0.474125,0.474125,0.00000,0.000000,0.000000
3,0.377964,0.377964,0.000000,0.00000,0.377964,0.377964,0.00000,0.00000,0.377964,0.000000,...,0.377964,0.377964,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.447214,...,0.000000,0.000000,0.000000,0.000000,0.447214,0.000000,0.000000,0.00000,0.447214,0.447214


## Exemple de classification

A partir du moment où nous disposons d'une matrice, nous pouvons appliquer toutes les approches que nous avons vu précédement.  
Nous illustrons au travers d'un exemple de classification multiclasse. Ce dernier est tiré de "The 20 newsgroups text dataset". Il s'agit d'un jeu de données de 20 newsgroups comprenant à peu près 18000 news sur 20 sujets différents. 

Ce jeu de données est disponible sous scikit learn qui propose des fonctions pour le manipuler : https://scikit-learn.org/stable/datasets/index.html#newsgroups-dataset  

fetch_20newsgroups permet de charger le fichier. Il est possible de récupérer un jeu d'entrainement, de test ou l'ensemble. 

** Présentation du jeu de données **


In [12]:
from sklearn.datasets import fetch_20newsgroups
news = fetch_20newsgroups(subset='all')
from pprint import pprint
print ("liste des topics \n")
pprint(list(news.target_names))

liste des topics 

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


** Téléchargement d'une partie des topics **

In [13]:
categories = ['alt.atheism', 'talk.religion.misc',
              'rec.sport.hockey','comp.graphics', 'sci.space']

news = fetch_20newsgroups(subset='all',
                          categories=categories)

print ("Taille du jeu de données\n")
print (news.filenames.shape)

print ("Un exemple de données\n")
print (news.data[5], '\n TOPIC : ',news.target[5],
       '\n*******************')



Taille du jeu de données

(4386,)
Un exemple de données

From: LMARSHA@cms.cc.wayne.edu (Laurie Marshall)
Subject: Re: WHERE ARE THE DOUBTERS NOW?  HMM?
Article-I.D.: cms.16BA79DBA.LMARSHA
Organization: Wayne State University, Detroit MI  U.S.A.
Lines: 22
NNTP-Posting-Host: cms.cc.wayne.edu

In article <1993Apr4.051942.27095@ramsey.cs.laurentian.ca>
maynard@ramsey.cs.laurentian.ca (Roger Maynard) writes:
 
>
>For those of you who can only decide which team is best after you have
>seen the standings:
>
>TOR  42 25 11  95   .609
>CHI  42 25 11  95   .609
>DET  44 28  9  97   .599
>VAN  41 28  9  91   .583
>
>No team in the Campbell Conference has a better record than Toronto.
 
  That's true, but according to your stats, Chicago has just as good a
record as Toronto.  It's interesting that you should list Toronto ahead
of Chicago.
 
 Laurie Marshall
 Wayne State University
 Detroit, Michigan
 Go Wings!!
 
 TOPIC :  2 
*******************


** Nettoyage des données **

In [14]:

def clean_news (data):
    for i in range(len(data)):
        #print (i)
        data[i]=clean_text(data[i])
    return data    
    


In [15]:
news.data=clean_news(news.data)  

** Vectorisation **

In [16]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(news.data)



** X et y **

In [17]:
#Specification des variables à prédire et de la classe X et y
X = vectors.toarray()

y = news.target


** Création du jeu d'apprentissage et de test **

In [18]:
from sklearn.model_selection import train_test_split 

validation_size=0.3 #30% du jeu de données pour le test

testsize= 1-validation_size
seed=30
X_train,X_test,y_train,y_test=train_test_split(X, 
                                               y, 
                                               train_size=validation_size, 
                                               random_state=seed,
                                               test_size=testsize)



** Utilisation du classifieur **

In [19]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from time import time

seed=7
k_fold = KFold(n_splits=10, shuffle=True, random_state=seed)
clf = GaussianNB()

scoring = 'accuracy'
t0 = time()
score = cross_val_score(clf, X, y, cv=k_fold, scoring=scoring)
print("Réalisé en %0.3fs" % (time() - t0))

print('Les différentes accuracy pour les 10 évaluations sont : \n',
      score,'\n')
print ('Accuracy moyenne : ',score.mean(), 
       ' standard deviation', score.std())

Réalisé en 166.380s
Les différentes accuracy pour les 10 évaluations sont : 
 [0.94305239 0.9498861  0.94305239 0.94077449 0.93394077 0.93621868
 0.93835616 0.94292237 0.91552511 0.95205479] 

Accuracy moyenne :  0.939578327664576  standard deviation 0.009600959296048284


** Mise en place d'un pipeline **

In [20]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix
from time import time
from sklearn.metrics import classification_report

pipeline = Pipeline([('vect', TfidfVectorizer()),
                ('clf', SGDClassifier(loss='hinge', 
                                      penalty='l2',
                                      alpha=1e-3, 
                                      random_state=42, 
                                      max_iter=5, tol=None)),
               ])




X=news.data
y=news.target


X_train,X_test,y_train,y_test=train_test_split(X, 
                                               y, 
                                               train_size=validation_size, 
                                               random_state=seed,
                                               test_size=testsize)


t0 = time()
pipeline.fit(X_train, y_train)
print("Fit réalisé en %0.3fs" % (time() - t0))

t0 = time()
result = pipeline.predict(X_test)
print("Prédiction réalisée en %0.3fs" % (time() - t0))

print('\n accuracy:',accuracy_score(result, y_test),'\n')




conf = confusion_matrix(y_test, result)
print ('\n matrice de confusion \n',conf)



print ('\n',classification_report(y_test, result))



/Users/pascalponcelet/Desktop/Sicki-learn/Tools/tools/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


Fit réalisé en 0.532s
Prédiction réalisée en 0.603s

 accuracy: 0.9397590361445783 


 matrice de confusion 
 [[510   6   1   5  24]
 [  1 665   4   4   1]
 [  0   9 722   0   1]
 [  1  21   4 653   1]
 [ 82   9   3   8 336]]

               precision    recall  f1-score   support

           0       0.86      0.93      0.89       546
           1       0.94      0.99      0.96       675
           2       0.98      0.99      0.98       732
           3       0.97      0.96      0.97       680
           4       0.93      0.77      0.84       438

   micro avg       0.94      0.94      0.94      3071
   macro avg       0.94      0.93      0.93      3071
weighted avg       0.94      0.94      0.94      3071



** Mise en place d'un gridsearch avec pipeline pour rechercher le meilleur classifieur **

Dans cette section nous intégrons un pipeline complet : lancement du TfidfVectorizer , utilisation de deux classifieurs (DecisionTreeClassifier et SGDClassifier) avec les hyperparamètres  pour évaluer le meilleur via GridSearchCV. Attention le processus de gridsearch est très long.

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from time import time
from sklearn.svm import SVC
import pickle


# Specification des pipelines
# programmation à optimiser par une fonction :)
pipeline_SGDC = Pipeline([('tfidf', TfidfVectorizer()),
                    ('clf', SGDClassifier())])


parameters_SGDC = [
    {'clf__max_iter': (5,),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet')}
]

pipeline_DT = Pipeline([('tfidf', TfidfVectorizer()),
                   ('clf', DecisionTreeClassifier())])


#param_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
param_range = [1, 5, 8, 10]
parameters_DT = [
    {'clf__min_samples_leaf': param_range,
        'clf__criterion': ['gini', 'entropy'],
        'clf__max_depth': param_range,
        'clf__min_samples_split': param_range[1:]}
]




X=news.data
y=news.target


X_train,X_test,y_train,y_test=train_test_split(X, 
                                               y, 
                                               train_size=validation_size, 
                                               random_state=seed,
                                               test_size=testsize)

# Creation des GridSearchCV avec les pipelines spécifiques

gs_SGDC = GridSearchCV(pipeline_SGDC, 
                       parameters_SGDC, 
                       cv=3,
                       n_jobs=-1, 
                       scoring='accuracy')


gs_DT = GridSearchCV(pipeline_DT, 
                     parameters_DT, 
                     cv=3,
                     n_jobs=-1, 
                     scoring='accuracy')



grids = [gs_SGDC, gs_DT]
grid_dict={0:'Linear classifiers', 1:'Decision Tree'}

best_acc = 0.0
best_clf = 0.0
best_gs = ''

for idx,gs in enumerate(grids):
    print('\nClassifier: %s' % grid_dict[idx])
    t0 = time()
    gs.fit(X_train, y_train)
    print("Fit réalisé en %0.3fs" % (time() - t0))

    print('Meilleurs paramètres : %s' % gs.best_params_)

    print("Meilleur score d'accuracy sur l'entrainement: %.3f" % gs.best_score_)
    # Prediction sur le jeu de test avec les meilleurs paramètres
    t0 = time()
    result = gs.predict(X_test)
    print("Prédiction réalisée en %0.3fs" % (time() - t0))
    
    print("Score d'accuracy pour les meilleurs paramètres sur jeu de test : %.3f"  % accuracy_score(y_test, result))

    print ('\n matrice de confusion \n',confusion_matrix(y_test, result))

    print ('\n',classification_report(y_test, result))
    
    #Modele avec la meilleure accuracy sur le jeu de test
    if accuracy_score(y_test, result) > best_acc:
        best_acc = accuracy_score(y_test, result)
        best_gs = gs
        best_clf = idx
        
        
        
print('\nClassifier avec la meilleur accuracy sur le jeu de test\n',
      grid_dict[best_clf])        




Classifier: Linear classifiers


/Users/pascalponcelet/Desktop/Sicki-learn/Tools/tools/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


Fit réalisé en 8.424s
Meilleurs paramètres : {'clf__alpha': 1e-06, 'clf__max_iter': 5, 'clf__penalty': 'l2'}
Meilleur score d'accuracy sur l'entrainement: 0.913
Prédiction réalisée en 0.682s
Score d'accuracy pour les meilleurs paramètres sur jeu de test : 0.933

 matrice de confusion 
 [[495   5   0   5  41]
 [  3 653  13   4   2]
 [  1  10 716   3   2]
 [  7  17   8 643   5]
 [ 58   8   8   7 357]]

               precision    recall  f1-score   support

           0       0.88      0.91      0.89       546
           1       0.94      0.97      0.95       675
           2       0.96      0.98      0.97       732
           3       0.97      0.95      0.96       680
           4       0.88      0.82      0.84       438

   micro avg       0.93      0.93      0.93      3071
   macro avg       0.93      0.92      0.92      3071
weighted avg       0.93      0.93      0.93      3071


Classifier: Decision Tree
Fit réalisé en 108.210s
Meilleurs paramètres : {'clf__criterion': 'entropy', 'c

** Recupération du meilleur classifieur avec ses paramètres **

Une fois le résultat obtenu, il est possible de récupérer le meilleur classifieur et ses paramètres. Ici il s'agit de Linear Classifier (SGDClassifier). Dans la suite nous montrons comment relancer une classification via un pipeline et une sauvegarde. Attention ici dans le pipeline nous ajoutons le clean_text (cf remarques précédentes) et nous sauvegardons le modèle pour pouvoir l'utiliser après avec d'autres données.   

Le fait de mettre le clean_text dans le pipeline permet lors de la sauvegarde de celui-ci de pouvoir lorsqu'il y a de nouvelles données de les relancer dans le pipeline (les donnés récupéreront dont la matrice du TfidfVectorizer et les pré-traitements associés). 

In [23]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix
from time import time
from sklearn.metrics import classification_report


#Recupération des données pour l'exemple
#et partir proprement
categories = ['alt.atheism', 'talk.religion.misc',
              'rec.sport.hockey','comp.graphics', 'sci.space']

news = fetch_20newsgroups(subset='all',
                          categories=categories)





pipeline = Pipeline([('vect', TfidfVectorizer(preprocessor=clean_text)),
                ('clf', SGDClassifier(loss='hinge', 
                                      penalty='l2',
                                      alpha=1e-05, 
                                      random_state=42, 
                                      max_iter=5, 
                                      tol=None)),
               ])




X=news.data
y=news.target


X_train,X_test,y_train,y_test=train_test_split(X, 
                                               y, 
                                               train_size=validation_size, 
                                               random_state=seed,
                                               test_size=testsize)


t0 = time()
print ("Lancement du fit \n")
pipeline.fit(X_train, y_train)
print("Fit réalisé en %0.3fs" % (time() - t0))

t0 = time()
print ("Lancement de la prédiction \n")
result = pipeline.predict(X_test)
print("Prédiction réalisée en %0.3fs" % (time() - t0))

print('\n accuracy:',accuracy_score(result, y_test),'\n')

conf = confusion_matrix(y_test, result)
print ('\n matrice de confusion \n',conf)



print ('\n',classification_report(y_test, result))

print("\nSauvegarde du pipeline grid search") 
filename = 'thebestone.pkl'
pickle.dump(pipeline, open(filename, 'wb'))



Lancement du fit 



/Users/pascalponcelet/Desktop/Sicki-learn/Tools/tools/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


Fit réalisé en 92.018s
Lancement de la prédiction 

Prédiction réalisée en 211.909s

 accuracy: 0.9296646043633996 


 matrice de confusion 
 [[495   5   7   8  31]
 [  4 647  12   9   3]
 [  0   7 723   1   1]
 [  5  20  13 637   5]
 [ 64   8   9   4 353]]

               precision    recall  f1-score   support

           0       0.87      0.91      0.89       546
           1       0.94      0.96      0.95       675
           2       0.95      0.99      0.97       732
           3       0.97      0.94      0.95       680
           4       0.90      0.81      0.85       438

   micro avg       0.93      0.93      0.93      3071
   macro avg       0.92      0.92      0.92      3071
weighted avg       0.93      0.93      0.93      3071


Sauvegarde du pipeline grid search


** Utilisation de nouvelles données **  

L'objectif ici est d'utiliser de nouvelles données à partir du modèle appris. Lors de la sauvegarde le pipeline entier a été sauvegardé. Cela implique que lorsque l'on va vouloir prédire les prétraitements du pipeline vont être appliqués.

In [25]:
import pickle
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print ("Chargement du modèle \n")
filename = 'thebestone.pkl'
clf_loaded = pickle.load(open(filename, 'rb'))


print ("A partir d'un nouveau texte\n")
print ("Utilisation d'un texte de 20newsgroup\n")
categories = ['alt.atheism', 'talk.religion.misc',
              'rec.sport.hockey','comp.graphics', 'sci.space']

news = fetch_20newsgroups(subset='all',
                          categories=categories)

print ("Sélection aléatoire de 20 documents \n")
from random import randint
samples=[]
samples_result=[]
sample_new=[]
for i in range(1,20):
    val=randint(1,4385)
    sample_new.append(val)
    samples.append(news.data[val])
    samples_result.append(news.target[val])
    
print ("Prédiction des news séléctionnées\n")    
    
 

result = clf_loaded.predict(samples)

print ("Valeurs réelles vs. valeurs prédites\n") 
for i in range(len(result)):
    print ("News : ",sample_new[i], 
           "\t réelle ", 
           samples_result[i], 
           " prédite ",
           result [i])

Chargement du modèle 

A partir d'un nouveau texte

Pour l'exemple utilisation d'un texte de 20newsgroup

Sélection aléatoire de 20 documents 

Prédiction des news séléctionnées

Valeurs réelles vs. valeurs prédites

News :  906 	 réelle  2  prédite  2
News :  3141 	 réelle  3  prédite  3
News :  2906 	 réelle  4  prédite  0
News :  693 	 réelle  2  prédite  2
News :  3454 	 réelle  1  prédite  1
News :  2993 	 réelle  3  prédite  3
News :  4080 	 réelle  3  prédite  3
News :  4042 	 réelle  1  prédite  1
News :  1520 	 réelle  2  prédite  2
News :  2277 	 réelle  2  prédite  2
News :  1769 	 réelle  3  prédite  3
News :  3359 	 réelle  2  prédite  2
News :  2642 	 réelle  1  prédite  1
News :  4247 	 réelle  1  prédite  1
News :  1871 	 réelle  0  prédite  0
News :  3921 	 réelle  1  prédite  1
News :  976 	 réelle  3  prédite  3
News :  1132 	 réelle  3  prédite  3
News :  2475 	 réelle  2  prédite  2
